In [21]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas

# Load your World Happiness Index data
df = pd.read_csv('World Happiness Report.csv')

# Ensure columns are appropriately formatted (e.g., remove spaces)
df.columns = df.columns.str.strip()


In [22]:
def create_happiness_plot(df):
    plot = df.hvplot.line(x='Year', y='Life Ladder', by='Country Name', legend='top_left', title='Happiness Plot')
    return plot

def create_happiness_table(df):
    table = df[['Country Name', 'Year', 'Life Ladder']].pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, sizing_mode='stretch_width')
    return table

def create_happiness_vs_gdp_scatter(df):
    scatter_plot = df.hvplot.scatter(x='Log GDP Per Capita', y='Life Ladder', by='Country Name', legend='top_left', title='Happiness vs GDP Scatter Plot')
    return scatter_plot

def create_happiness_metrics_bar_plot(df):
    metrics_pipeline = create_happiness_metrics_pipeline(df)
    bar_plot = metrics_pipeline.hvplot.bar(x='Country Name', y=['Social Support', 'Freedom To Make Life Choices', 'Generosity'],
                                           title='Happiness Metrics by Country', width=700, height=400)
    return bar_plot

def create_happiness_metrics_pipeline(df):
    pipeline = df.groupby(['Country Name', 'Year']).agg({
        'Life Ladder': 'mean',
        'Log GDP Per Capita': 'mean',
        'Social Support': 'mean',
        'Healthy Life Expectancy At Birth': 'mean',
        'Freedom To Make Life Choices': 'mean',
        'Generosity': 'mean',
        'Perceptions Of Corruption': 'mean',
        'Positive Affect': 'mean',
        'Negative Affect': 'mean',
        'Confidence In National Government': 'mean'
    }).reset_index()
    return pipeline


In [23]:
# Year slider widget
year_slider = pn.widgets.IntSlider(name='Year', start=df['Year'].min(), end=df['Year'].max(), step=1, value=df['Year'].min())

# Regional indicator selector widget
regions = df['Regional Indicator'].unique()
region_selector = pn.widgets.Select(name='Regional Indicator', options=list(regions), value=regions[0])

# Callback function to update plots based on year and region
def update_plots(event):
    year = year_slider.value
    region = region_selector.value
    
    # Filter dataframe based on selected year and region
    filtered_df = df[(df['Year'] == year) & (df['Regional Indicator'] == region)]
    
    # Update plots
    happiness_plot.object = create_happiness_plot(filtered_df)
    happiness_table.object = create_happiness_table(filtered_df)
    happiness_vs_gdp_scatter.object = create_happiness_vs_gdp_scatter(filtered_df)
    happiness_metrics_bar_plot.object = create_happiness_metrics_bar_plot(filtered_df)

# Link callback to slider and selector events
year_slider.param.watch(update_plots, 'value')
region_selector.param.watch(update_plots, 'value')


Watcher(inst=Select(name='Regional Indicator', options=['South Asia', ...], value='South Asia'), cls=<class 'panel.widgets.select.Select'>, fn=<function update_plots at 0x000002591CA2C550>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [24]:
# Initialize with data for the initial year and region
initial_year = year_slider.value
initial_region = region_selector.value
filtered_df = df[(df['Year'] == initial_year) & (df['Regional Indicator'] == initial_region)]

# Initialize plots
happiness_plot = create_happiness_plot(filtered_df)
happiness_table = create_happiness_table(filtered_df)
happiness_vs_gdp_scatter = create_happiness_vs_gdp_scatter(filtered_df)
happiness_metrics_bar_plot = create_happiness_metrics_bar_plot(filtered_df)


In [25]:
# Sidebar with widgets
sidebar = pn.Column(
    year_slider,
    region_selector
)

# Main section with plots
main_section = pn.Column(
    pn.Row(happiness_plot, happiness_table),
    pn.Row(happiness_vs_gdp_scatter, happiness_metrics_bar_plot)
)

# Combine sidebar and main section into a Panel layout
dashboard = pn.Column(
    pn.Row(
        sidebar,
        main_section
    )
)


In [26]:
# Display the dashboard
dashboard.servable()


Column
    [0] Row
        [0] Column
            [0] IntSlider(end=2022, name='Year', start=2005, value=2005)
            [1] Select(name='Regional Indicator', options=['South Asia', ...], value='South Asia')
        [1] Column
            [0] Row
                [0] HoloViews(NdOverlay, height=300, sizing_mode='fixed', width=700)
                [1] Tabulator(page_size=10, pagination='remote', sizing_mode='stretch_width', value=     Country N...)
            [1] Row
                [0] HoloViews(NdOverlay, height=300, sizing_mode='fixed', width=700)
                [1] HoloViews(Bars, height=400, sizing_mode='fixed', width=700)